In [2]:
import pandas as pd
import os
from itertools import chain
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
directory = '../Data'  # Directory where the pickle files are stored
pickle_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pickle')]

dataframes = []
for file in pickle_files:
    s = pd.read_pickle(file)
    flattened_list = list(chain.from_iterable(s))

    # Create a DataFrame
    df = pd.DataFrame(flattened_list, columns=['ID', 'DateTime'])
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)


In [4]:
counts=combined_df.groupby('ID').count().sort_values(by='ID').reset_index()
total_counts = counts['DateTime'].sum()


# Calculate the density for each 'ID'
density = counts['DateTime'] / total_counts


counts['density'] = density
grid_30x30 = np.zeros((30, 30))

# Iterate over the DataFrame and assign densities to the grid
for index, row in counts.iterrows():
    # Ensure the ID is within the valid range (0 to 899)
    if 0 <= row['ID'] < 900:
        row_idx = row['ID'] // 30  # Calculate row index
        col_idx = row['ID'] % 30   # Calculate column index
        if grid_30x30[int(row_idx), int(col_idx)]==0:
            grid_30x30[int(row_idx), int(col_idx)] = row['density']
        else:
            print("Duplicate")
            print(int(row_idx), int(col_idx))

In [5]:
def update_density_grid(current_time,combined_df):
    # Code to update the density grid every 10 days
    subset= combined_df[combined_df.DateTime<current_time+pd.Timedelta(days=10)]
    
    counts=subset.groupby('ID').count().sort_values(by='ID').reset_index()
    total_counts = counts['DateTime'].sum()
    

    # Calculate the density for each 'ID'
    density = counts['DateTime'] / total_counts


    counts['density'] = density
    counts
    grid_30x30 = np.zeros((30, 30))

    # Iterate over the DataFrame and assign densities to the grid
    for index, row in counts.iterrows():
        # Ensure the ID is within the valid range (0 to 899)
        if 0 <= row['ID'] < 900:
            row_idx = row['ID'] // 30  # Calculate row index
            col_idx = row['ID'] % 30   # Calculate column index
            if grid_30x30[int(row_idx), int(col_idx)]==0:
                grid_30x30[int(row_idx), int(col_idx)] = row['density']
            else:
                print("Duplicate")
                print(int(row_idx), int(col_idx))
    
    return grid_30x30

In [6]:
def aggregate_grid(grid, block_size):
    # Aggregate the grid into larger blocks
    num_rows, num_cols = grid.shape
    aggregated_grid = np.zeros((num_rows // block_size, num_cols // block_size))

    for i in range(0, num_rows, block_size):
        for j in range(0, num_cols, block_size):
            aggregated_grid[i // block_size, j // block_size] = np.sum(grid[i:i+block_size, j:j+block_size])
    
    return aggregated_grid

def solve_p_median(grid, p, potential_facilities):
    # Create a model
    m = gp.Model("p_median")
    num_rows, num_cols = grid.shape

    # Decision Variables
    x = m.addVars(num_rows, num_cols, vtype=GRB.BINARY, name="x")

    # Objective Function: Minimize the weighted sum of distances
    m.setObjective(gp.quicksum(grid[i, j] * (abs(i-k) + abs(j-l)) * x[k, l]
                               for i in range(num_rows) for j in range(num_cols)
                               for k, l in potential_facilities), GRB.MINIMIZE)

    # Constraints

    # Constraint 1: Exactly p facilities
    m.addConstrs((x[k, l] <= 1 for k, l in potential_facilities), "facility_limit")
    m.addConstr(gp.quicksum(x[k, l] for k, l in potential_facilities) == p, "facility_count")

    # Solve the model
    m.optimize()

    # Extract and print the solution
    if m.status == GRB.OPTIMAL:
        solution_x = m.getAttr('x', x)
        print("Facility Locations:")
        for k, l in potential_facilities:
            if solution_x[k, l] > 0.5:
                print(f"Facility at cell ({k}, {l})")
    else:
        print("No optimal solution found")

# Example Usage
original_grid = grid_30x30 
aggregated_grid = aggregate_grid(original_grid, block_size=1)

potential_facilities = [(i, j) for i in range(aggregated_grid.shape[0]) for j in range(aggregated_grid.shape[1]) if (i % 1 == 0 and j % 1 == 0)]

# Solve the problem
solve_p_median(aggregated_grid, p=3, potential_facilities=potential_facilities)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-30
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x42b39a93
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 52.6905445
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 15.6119 52.6905 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.561191349064e+01, best bound 1.561191349064e+01, ga

In [7]:
# Main loop
current_time=combined_df.DateTime.min()
while (current_time<combined_df.DateTime.max()):
    density_grid = update_density_grid(current_time,combined_df)
    solve_p_median(density_grid, 3, potential_facilities)
    current_time+=pd.Timedelta(days=10)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x819d5d56
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 53.0555932
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 22.3969 53.0556 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.239691003256e+01, best bound 2.239691003256e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 16)
Facility at cell (12, 17)
Facility 

In [8]:
solve_p_median(aggregated_grid, p=4, potential_facilities=potential_facilities)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x67c74962
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 76.1688852
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 21.0806 76.1689 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.108062889058e+01, best bound 2.108062889058e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility 

In [10]:
solve_p_median(aggregated_grid, p=5, potential_facilities=potential_facilities)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x7ce28d0e
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 87.8165490
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 26.6631 87.8165 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.666306621066e+01, best bound 2.666306621066e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility 

In [11]:
solve_p_median(aggregated_grid, p=6, potential_facilities=potential_facilities)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x6d980324
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 100.9113747
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 32.2887 100.911 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.228870408188e+01, best bound 3.228870408188e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility

In [12]:
solve_p_median(aggregated_grid, p=7, potential_facilities=potential_facilities)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x30afe6ed
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 117.3652535
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 37.9605 117.365 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.796047043771e+01, best bound 3.796047043771e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility

In [13]:
solve_p_median(aggregated_grid, p=8, potential_facilities=potential_facilities)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 901 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x76531244
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 122.9476909
Presolve removed 901 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 43.7199 122.948 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.371989272445e+01, best bound 4.371989272445e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility